In [ ]:
import numpy as np
np.random.seed(108)

import keras
from keras.datasets import mnist
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization, Activation
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from math import sqrt

from IPython.display import SVG

import os
import smtplib
from email.mime.text import MIMEText

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def data():

    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    
    X_train = (X_train - 255/2.0) / 255
    X_test = (X_test - 255/2.0) / 255

    X_train = X_train.reshape(60000, 28, 28, 1)
    X_test = X_test.reshape(10000, 28, 28, 1)
    
    y_train = keras.utils.to_categorical(y_train)
    y_test = keras.utils.to_categorical(y_test)
    
    return X_train, y_train, X_test, y_test

In [ ]:
def create_model(X_train, y_train, X_test, y_test):

    datagen = ImageDataGenerator(rotation_range=15,
                                 width_shift_range=2.0/28,
                                 height_shift_range=2.0/28
                                )
    
    epochs = 10 
    lr = (1.234e-3)
    optimizer = Adam(lr=lr)
    
    main_input = Input(shape= (28,28,1), name='main_input')
    
    sub_models = []
    
    for i in range(5):

        x = Conv2D(32, kernel_size=(3,3), strides=1)(main_input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPool2D(pool_size=2)(x)

        x = Conv2D(64, kernel_size=(3,3), strides=1)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPool2D(pool_size=2)(x)

        x = Conv2D(64, kernel_size=(3,3), strides=1)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Flatten()(x)

        x = Dense(1024)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.1)(x)

        x = Dense(256)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.4)(x)
        
        x = Dense(10, activation='softmax')(x)
        
        sub_models.append(x)
    
    x = keras.layers.average(sub_models)
    
    main_output = keras.layers.average(sub_models)
    
    model = Model(inputs=[main_input], outputs=[main_output])
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=optimizer)
    
    print(model.summary())
        
    filepath="weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    tensorboard = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
    callbacks = [checkpoint, tensorboard]

    model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),
                        steps_per_epoch=len(X_train) / 128,
                        epochs=epochs,
                        callbacks=callbacks,
                        verbose=1,
                        validation_data=(X_test, y_test))
    
    print('Test accuracy:', acc)
    return model

In [ ]:
X_train, y_train, X_test, y_test = data()
model = create_model(X_train, y_train, X_test, y_test)

In [ ]:
from keras.models import load_model
model = load_model("weights.best.hdf5")
score, acc = model.evaluate(X_test, y_test, verbose=0)
print(score, acc)